In [ ]:
#Librerias
import pandas as pd
import pyodbc
import schedule
import time
from datetime import datetime

In [3]:
# Configuración de la conexión a SQL Server
server = '34.135.222.123'
database = 'nbaplayers'
username = 'sqlserver'
password = 'Contraseña1'
driver = 'ODBC Driver 17 for SQL Server'

In [ ]:




# Función para la ingesta de datos
def ingest_data():
    try:
        print(f"Iniciando ingesta de datos: {datetime.now()}")

        # 1. Extracción: Leer datos del archivo CSV
        archivo_csv = 'C:\\nba_stats.csv.csv'
        df = pd.read_csv(archivo_csv)

        # 2. Transformación: Limpiar los datos (opcional según tus necesidades)
        df = df.dropna()  # Eliminar filas con valores NaN

        # 3. Conectar a la base de datos SQL Server e insertar los datos
        conexion = pyodbc.connect(
            f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
        )
        cursor = conexion.cursor()

        # Crear una sentencia SQL para insertar datos
        sql_insert = """
            INSERT INTO nba_stats (PLAYER_ID, PLAYER_NAME, NICKNAME, TEAM_ID, TEAM_ABBREVIATION, AGE, GP, W, L, W_PCT, MIN, FGM, FGA, FG_PCT, FG3M, FG3A, FG3_PCT, FTM, FTA, FT_PCT, OREB, DREB, REB, AST, TOV, STL, BLK, BLKA, PF, PFD, PTS, PLUS_MINUS, NBA_FANTASY_PTS, DD2, TD3, WNBA_FANTASY_PTS,
            GP_RANK, W_RANK, L_RANK, W_PCT_RANK, MIN_RANK, FGM_RANK, FGA_RANK, FG_PCT_RANK, FG3M_RANK, FG3A_RANK, FTM_RANK, FTA_RANK, FT_PCT_RANK, OREB_RANK, DREB_RANK, REB_RANK, AST_RANK, TOV_RANK, STL_RANK, BLK_RANK, BLKA_RANK, PF_RANK, PFD_RANK, PTS_RANK, PLUS_MINUS_RANK, NBA_FANTASY_PTS_RANK,
            DD2_RANK, TD3_RANK, WNBA_FANTASY_PTS_RANK)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Recorrer el DataFrame e insertar los datos
        for index, row in df.iterrows():
            cursor.execute(sql_insert, (
                row['PLAYER_ID'], row['PLAYER_NAME'], row['NICKNAME'], row['TEAM_ID'], row['TEAM_ABBREVIATION'], 
                row['AGE'], row['GP'], row['W'], row['L'], row['W_PCT'], row['MIN'], row['FGM'], row['FGA'], row['FG_PCT'], 
                row['FG3M'], row['FG3A'], row['FG3_PCT'], row['FTM'], row['FTA'], row['FT_PCT'], row['OREB'], row['DREB'], 
                row['REB'], row['AST'], row['TOV'], row['STL'], row['BLK'], row['BLKA'], row['PF'], row['PFD'], row['PTS'], 
                row['PLUS_MINUS'], row['NBA_FANTASY_PTS'], row['DD2'], row['TD3'], row['WNBA_FANTASY_PTS'], row['GP_RANK'], 
                row['W_RANK'], row['L_RANK'], row['W_PCT_RANK'], row['MIN_RANK'], row['FGM_RANK'], row['FGA_RANK'], row['FG_PCT_RANK'],
                row['FG3M_RANK'], row['FG3A_RANK'], row['FTM_RANK'], row['FTA_RANK'], row['FT_PCT_RANK'], row['OREB_RANK'],
                row['DREB_RANK'], row['REB_RANK'], row['AST_RANK'], row['TOV_RANK'], row['STL_RANK'], row['BLK_RANK'], row['BLKA_RANK'], 
                row['PF_RANK'], row['PFD_RANK'], row['PTS_RANK'], row['PLUS_MINUS_RANK'], row['NBA_FANTASY_PTS_RANK'], row['DD2_RANK'], 
                row['TD3_RANK'], row['WNBA_FANTASY_PTS_RANK']
            ))

        # Confirmar la transacción
        conexion.commit()

        cursor.close()
        conexion.close()

        print(f"Ingesta completada exitosamente: {datetime.now()}")

    except Exception as e:
        print(f"Error durante la ingesta: {e}")

# Función para programar la tarea diaria
def schedule_job():
    # Programar la ingesta de datos
    schedule.every().day.at("12:00").do(ingest_data)

    print("Automatización de la ingesta de datos iniciada. Esperando próximas ejecuciones...")

    # Bucle infinito para mantener la tarea programada corriendo
    while True:
        schedule.run_pending()
        time.sleep(60)  # Verificar cada minuto si hay tareas pendientes


# Ejecutar el proceso de automatización
if __name__ == "__main__":
    schedule_job()
